In [1]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class CustomKMeans:
    def __init__(self, n_clusters, max_iter=300, tol=1e-4, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.centroids = None
        self.labels = None

    def fit(self, X):
        np.random.seed(self.random_state)
        # Initialize centroids using kmeans++
        self.centroids = self._initialize_centroids(X)

        for _ in range(self.max_iter):
            # Assign each data point to the nearest centroid
            labels = self._assign_labels(X)
            # Update centroids based on the mean of data points in each cluster
            new_centroids = self._update_centroids(X, labels)
            # Check if centroids have converged
            if np.linalg.norm(new_centroids - self.centroids) < self.tol:
                break
            self.centroids = new_centroids
            self.labels = labels

    def predict(self, X):
        # Assign each data point to the nearest centroid
        return self._assign_labels(X)

    def _initialize_centroids(self, X):
        centroids = [X[np.random.choice(X.shape[0])]]
        while len(centroids) < self.n_clusters:
            dist_sq = np.array([min([np.linalg.norm(x - c)**2 for c in centroids]) for x in X])
            probs = dist_sq / dist_sq.sum()
            cumulative_probs = probs.cumsum()
            r = np.random.rand()
            i = np.searchsorted(cumulative_probs, r)
            centroids.append(X[i])
        return np.array(centroids)

    def _assign_labels(self, X):
        # Assign each data point to the nearest centroid
        return np.argmin(np.linalg.norm(X[:, np.newaxis] - self.centroids, axis=2), axis=1)

    def _update_centroids(self, X, labels):
        # Update centroids based on the mean of data points in each cluster
        new_centroids = np.array([X[labels == i].mean(axis=0) if np.sum(labels == i) > 0 else self.centroids[i] for i in range(self.n_clusters)])
        return new_centroids


In [5]:
import pandas as pd
# Load data
print("Loading data...")
movie_data = pd.read_csv('/content/drive/MyDrive/rsdats/movies.csv')
user_data = pd.read_csv('/content/drive/MyDrive/rsdats/users.csv')
rating_data = pd.read_csv('/content/drive/MyDrive/rsdats/ratings.csv')

# Merge rating data with movie data
print("Merging rating data with movie data...")
merged_data = pd.merge(rating_data, movie_data, left_on='MovieID', right_on='ID')

# Merge with user data using UserID
print("Merging with user data using UserID...")
merged_data = pd.merge(merged_data, user_data, on='UserID')

# Pivot table of user ratings with movie titles as columns
print("Creating user-item matrix...")
user_ratings = merged_data.pivot_table(index='UserID', columns='Title', values='Rating').fillna(0)

Loading data...
Merging rating data with movie data...
Merging with user data using UserID...
Creating user-item matrix...


In [10]:
import numpy as np
from tqdm import tqdm

class CustomSVD:
    def __init__(self, A, num_components=None):
        print("Computing SVD...")
        self.U, self.S, self.Vt = self.svd(A, num_components)

    def svd(self, A, num_components=None):
        m, n = A.shape

        # Compute A^T * A
        print("Compute A^T * A")
        ATA = A.T.dot(A)

        # Compute eigenvalues and eigenvectors of A^T * A
        print("Compute eigenvalues and eigenvectors of A^T * A")
        eigenvalues, Vt = self.power_iteration(ATA)

        # Sort eigenvalues in descending order
        print("Sort eigenvalues in descending order")
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        Vt = Vt[:, idx]

        # Determine the number of components
        if num_components is None:
            num_components = min(m, n)

        # Truncate singular values and vectors
        singular_values = np.sqrt(eigenvalues)[:num_components]
        U = np.zeros((m, num_components))
        for i in tqdm(range(num_components), desc="Computing U"):
            u = np.dot(A, Vt[:, i]) / singular_values[i]
            for j in range(i):
                u -= np.dot(U[:, j], u) * U[:, j]
            U[:, i] = u / np.linalg.norm(u)

        return U, np.diag(singular_values), Vt.T[:, :num_components]

    def power_iteration(self, A, max_iter=2, tol=1e-6):
        n = A.shape[0]
        eigenvalues = np.zeros(n)
        eigenvectors = np.zeros((n, n))

        for i in tqdm(range(n), desc="power itr"):
            # Set an initial guess for the eigenvector
            x = np.random.rand(n)
            x /= np.linalg.norm(x)

            # Iterative method to find eigenvalues and eigenvectors using Power Iteration
            for _ in range(max_iter):
                x_next = A.dot(x)
                eigenvalue = np.linalg.norm(x_next)
                x_next /= eigenvalue

                # Check for convergence
                if np.linalg.norm(x_next - x) < tol:
                    break

                x = x_next

            # Set the computed eigenvalue and eigenvector
            eigenvalues[i] = eigenvalue
            eigenvectors[:, i] = x

            # Deflate the matrix
            A -= eigenvalue * np.outer(x, x)

        return eigenvalues, eigenvectors


In [11]:
# Apply SVD to reduce dimensions of the user ratings matrix
svd = CustomSVD(user_ratings.values)
U, S, Vt = svd.U, svd.S, svd.Vt

Computing SVD...
Compute A^T * A
Compute eigenvalues and eigenvectors of A^T * A


power itr: 100%|██████████| 3706/3706 [08:04<00:00,  7.65it/s]


Sort eigenvalues in descending order


Computing U: 100%|██████████| 3706/3706 [27:48<00:00,  2.22it/s]


In [12]:
U, S, Vt

(array([[ 0.0047134 , -0.00473099, -0.00686935, ...,  0.01681287,
         -0.01389041,  0.02526799],
        [ 0.00928017,  0.00166753, -0.00949428, ..., -0.00732309,
          0.00418976,  0.00076775],
        [ 0.00499941, -0.00010978, -0.01208486, ..., -0.00814072,
          0.03186233,  0.01105844],
        ...,
        [ 0.00138807, -0.00266405, -0.0017137 , ..., -0.01813244,
          0.00679553, -0.01281695],
        [ 0.00701449, -0.01993456,  0.00187645, ...,  0.00911832,
          0.00256742,  0.01683817],
        [ 0.01896282, -0.03955345, -0.00948862, ..., -0.00999643,
         -0.00326711,  0.00411221]]),
 array([[1891.93651079,    0.        ,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        [   0.        ,  589.70003056,    0.        , ...,    0.        ,
            0.        ,    0.        ],
        [   0.        ,    0.        ,  498.55157491, ...,    0.        ,
            0.        ,    0.        ],
        ...,
        [   0.    

In [15]:
U[0]

array([ 0.0047134 , -0.00473099, -0.00686935, ...,  0.01681287,
       -0.01389041,  0.02526799])

In [16]:
# Apply KMeans clustering to cluster users based on reduced features
kmeans = CustomKMeans(n_clusters=4)
kmeans.fit(U)

In [17]:
# Add cluster labels to user data
user_data['Cluster'] = kmeans.predict(U)

In [18]:
# New user ratings
new_user_ratings = {'Toy Story (1995)': 5, 'Jurassic Park (1993)': 4, 'Forrest Gump (1994)': 3}

# Create a DataFrame for the new user
new_user_data = pd.DataFrame(new_user_ratings, index=[0])

# Convert movie titles to columns
new_user_movies = pd.DataFrame(columns=user_ratings.columns)

# Merge new user data with the movie DataFrame
new_user_data_merged = pd.merge(new_user_movies, new_user_data, how='outer').fillna(0)

# Reorder the columns of new_user_data_merged to match the order of user_ratings
new_user_data_reordered = new_user_data_merged[user_ratings.columns]

In [19]:
# Transform the new user data using SVD
new_user_data_reduced = np.dot(new_user_data_reordered, Vt.T)

# Find the cluster of the new user
new_user_cluster = kmeans.predict(new_user_data_reduced)
print(new_user_cluster)

[3]


In [20]:
# Filter users in the same cluster
users_same_cluster = user_data[user_data['Cluster'] == new_user_cluster[0]]

# Get the IDs of users in the same cluster
user_ids_same_cluster = users_same_cluster['UserID']

# Filter ratings of users in the same cluster
ratings_same_cluster = merged_data[merged_data['UserID'].isin(user_ids_same_cluster)]

# Calculate the mean rating for each movie among users in the same cluster
mean_ratings = ratings_same_cluster.groupby('Title')['Rating'].mean().sort_values(ascending=False)

print(mean_ratings.head())

Title
Follow the Bitch (1998)                      5.0
Lured (1947)                                 5.0
Schlafes Bruder (Brother of Sleep) (1995)    5.0
Ulysses (Ulisse) (1954)                      5.0
Baby, The (1973)                             5.0
Name: Rating, dtype: float64
